In [1]:
# !pip install datasets
# !pip install evaluate
# ! pip install -U accelerate
# ! pip install -U transformers

In [1]:
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
from bs4 import BeautifulSoup
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import copy
from tqdm import tqdm
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np

Setting ds_accelerator to cuda (auto detect)


In [3]:
id2label = {0: "NORMAL", 1: "TOXIC"}
label2id = {"NORMAL": 0, "TOXIC": 1}

In [3]:
#tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-xlarge-mnli")
#model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-xlarge-mnli", num_labels=2, label2id=label2id, id2label=id2label, ignore_mismatched_sizes=True)
#model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-xlarge-mnli")

# Load model directly
# from transformers import AutoTokenizer, AutoModelForMaskedLM

# tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")
# model = AutoModelForMaskedLM.from_pretrained("FacebookAI/xlm-roberta-base")

In [14]:
tokenizer = AutoTokenizer.from_pretrained("/home/sslashinin/kovakimyan/diploma/deberta-xlarge-mnli/", local_files_only=True)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NORMAL', '1': 'TOXIC'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NORMAL', '1': 'TOXIC'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NORMAL', '1': 'TOXIC'}. The number of labels wil be overwritten to 2.


In [18]:
model = AutoModelForSequenceClassification.from_pretrained("/home/sslashinin/kovakimyan/diploma/deberta-xlarge-mnli/", local_files_only=True, ignore_mismatched_sizes=True)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NORMAL', '1': 'TOXIC'}. The number of labels wil be overwritten to 2.
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at /home/sslashinin/kovakimyan/diploma/deberta-xlarge-mnli/ and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 1024]) in the checkpoint and torch.Size([2, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
dataset = load_dataset("textdetox/multilingual_toxicity_dataset")

dataset

Found cached dataset parquet (/home/sslashinin/.cache/huggingface/datasets/textdetox___parquet/textdetox--multilingual_toxicity_dataset-c4d45e458288ed14/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/9 [00:00<?, ?it/s]

DatasetDict({
    am: Dataset({
        features: ['text', 'toxic'],
        num_rows: 5000
    })
    ar: Dataset({
        features: ['text', 'toxic'],
        num_rows: 5000
    })
    de: Dataset({
        features: ['text', 'toxic'],
        num_rows: 5000
    })
    en: Dataset({
        features: ['text', 'toxic'],
        num_rows: 5000
    })
    es: Dataset({
        features: ['text', 'toxic'],
        num_rows: 5000
    })
    hi: Dataset({
        features: ['text', 'toxic'],
        num_rows: 5000
    })
    ru: Dataset({
        features: ['text', 'toxic'],
        num_rows: 5000
    })
    uk: Dataset({
        features: ['text', 'toxic'],
        num_rows: 5000
    })
    zh: Dataset({
        features: ['text', 'toxic'],
        num_rows: 5000
    })
})

In [20]:
dataset["en"]

Dataset({
    features: ['text', 'toxic'],
    num_rows: 5000
})

In [21]:
def decontracted(phrase):
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def cleanPunc(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned

def clear_sentance(sentance):
    sentance= re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = cleanPunc(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    stop_words = set(stopwords.words('english'))
    stop_words.update(['zero','one','two','three','four','five','six','seven','eight','nine','ten','may','also','across','among','beside','however','yet','within'])
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in  stopwords.words('english'))
    return sentance.strip()

In [22]:
preprocessed_text = []
for sentance in tqdm(dataset["en"]['text']):
    preprocessed_text.append(clear_sentance(sentance))

  0%|          | 0/5000 [00:00<?, ?it/s]/tmp/job-1888621/ipykernel_223617/3189064709.py:23: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  sentance = BeautifulSoup(sentance, 'lxml').get_text()
100%|██████████| 5000/5000 [00:09<00:00, 523.68it/s]


In [23]:
new_dataset = copy.deepcopy(dataset["en"])

new_dataset = new_dataset.add_column("preprocessed_text", preprocessed_text)

In [24]:
en_dataset = copy.deepcopy(new_dataset)

en_dataset_split = en_dataset.train_test_split(test_size=0.3)

en_dataset_split = en_dataset_split.rename_column("toxic", "label")

en_dataset_split["train"] = en_dataset_split["train"].remove_columns("text")
en_dataset_split["test"] = en_dataset_split["test"].remove_columns("text")

en_dataset_split

DatasetDict({
    train: Dataset({
        features: ['label', 'preprocessed_text'],
        num_rows: 3500
    })
    test: Dataset({
        features: ['label', 'preprocessed_text'],
        num_rows: 1500
    })
})

In [25]:
def preprocess_function(examples):
    return tokenizer(examples["preprocessed_text"], truncation=True)

tokenized_dataset = en_dataset_split.map(preprocess_function, batched=True)

tokenized_dataset

Map:   0%|          | 0/3500 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'preprocessed_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3500
    })
    test: Dataset({
        features: ['label', 'preprocessed_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1500
    })
})

In [26]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [27]:
accuracy = evaluate.load("accuracy")

In [28]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [33]:
training_args = TrainingArguments(
    output_dir="deberta_xlarge_mnli_ft",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    #save_strategy="epoch",
    #load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.save_model()

/home/sslashinin/.conda/envs/kovakimyan/lib/python3.11/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.219914,0.949333
2,0.320400,0.246344,0.941333
3,0.169800,0.206414,0.956667
4,0.135400,0.220114,0.960000
5,0.061500,0.237834,0.957333


In [31]:
import torch

In [35]:
def run_model(text):
    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class_id = logits.argmax().item()
    print(model.config.id2label[predicted_class_id])

In [36]:
text = "Fuck you"
run_model(text)

text = "You act like a fool"
run_model(text)

text = "Today is sunny"
run_model(text)

TOXIC
TOXIC
NORMAL


wandb: Network error (ReadTimeout), entering retry loop.
